kali ini akan dibangun sebuah chain management untuk sistem LLM. Terdapat beberapa input yang akan menjadi pipeline untuk LLM. 

Saya akan mencoba mempersiapkan input dari data yang telah dimasukkan atau di-input pada chroamDB. 

Query dari user untuk LLM ini juga akan dijadikan sebagai query untuk mengambil data yang relevan dari ChroamDB

### Beberapa hal yang perlu di define
* Model LLM 
* Model Embedding untuk mengubah teks menjadi vector
* Memasukkan vector kedalam ChromaDB dengan langchain_chroma

#### Model LLM

In [1]:
from langchain_community.chat_models import ChatOllama

/Users/a/Programming/Langchain-Project/my-env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm_model = ChatOllama(
    model='llama3:8b',
    temperature=0
)

/var/folders/05/16tm42991snf6c27rs7nssdh0000gn/T/ipykernel_16296/2586603255.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  llm_model = ChatOllama(


#### Model Embeddings

In [3]:
# memanggil Indobert dari transformer
from transformers import BertTokenizer, AutoModel

tokenizer = BertTokenizer.from_pretrained("Indobenchmark/indobert-base-p1")
model = AutoModel.from_pretrained("indobenchmark/indobert-base-p1")

In [4]:
# membuat class model embdding
from typing import List 
from langchain_core.embeddings import Embeddings
import torch

class IndoBertEmbeddings(Embeddings):
    def __init__(self, model_name="indobenchmark/indobert-base-p1"):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.model.eval()


    def _generate_embedding(self, text: str) -> List[float]:
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

        with torch.no_grad():
            outputs = self.model(**inputs)

        # polling token menjadi satu vector kalimat
        token_embeddings = outputs.last_hidden_state

        # melakukan mean polling
        sentence_embeddings = token_embeddings.mean(dim=1)

        # konversi ke list python
        return sentence_embeddings.squeeze().tolist()
    

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return [self._generate_embedding(text) for text in texts]
    

    # metode untuk pencarian query pada chroma 
    def embed_query(self, text: str) -> List[float]:
        return self._generate_embedding(text)


### Feeding data to chromaDB dengan menggunakan model class Indobert Embeddings

In [5]:
from langchain_chroma import Chroma

In [6]:
embeddings = IndoBertEmbeddings()

In [7]:
vectore_store = Chroma(
    collection_name="chroma-with-langchain",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db"
)

### Dataset Section 

In [8]:
from langchain_core.documents import Document

data_teks = [Document(
    page_content="Presiden Indonesia sekarang adalah Prabowo Subianto dan wakil presidennya adalah Gibran Rakabuming Raka",
    metadata={"sumber":'wikipedia', "tahun":2025}
)]

In [9]:
vectore_store.add_documents(data_teks)

['e79c6fb6-180a-4f4f-8380-dcdb30acaec4']

Test Query atau proses Retrieve pada ChromaDB

In [10]:
query = "presiden dan wakil presidensekarang"

In [11]:
result = vectore_store.similarity_search(query=query, k=1)

In [12]:
for doc in result:
    print(doc.page_content)

Presiden pertama Indonesia adalah Ir. Soekarno.


Note. Proses Retreive belum sepenuhnya maksimal, karena pada proses diatas, proses retrieval belum maksimal memberikan data yang relevan. Hal ini nanti bisa di uji dengan retrieval generation testing

### Chains Template atau prompt dengan model LLM

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [52]:
prompt = ChatPromptTemplate.from_template(
    """Kamu adalah asisten chatbot yang membantu. 
    Tugasmu adalah menjawab pertanyaan berikut secara langsung dalam Bahasa Indonesia. 
    Meskipun pertanyaan diberikan dalam Bahasa Inggris atau bahasa lain, kamu WAJIB menjawabnya dalam Bahasa Indonesia.
    
    Pertanyaan: {input}
    Jawaban:"""
)

In [53]:
interface_user = "what is novelty"

In [54]:
chain = prompt | llm_model | StrOutputParser()

In [55]:
respone = chain.invoke({"input": interface_user})
print(respone)

Inovasi! Novelty dalam konteks ini berarti sesuatu yang baru, unik, dan tidak biasa. Dalam arti luas, novelty dapat merujuk pada gagasan, ide, atau produk yang belum pernah ada sebelumnya dan memiliki potensi untuk menjadi inovatif dan berbeda dari apa yang telah diketahui sebelumnya.
